In [ ]:
import numpy as np
import sys
import cv2
import random
import os
import numpy as np
import json
from matplotlib import pyplot as plt
from PIL import Image
from pycococreatortools import pycococreatortools
from IPython import embed
from tqdm import tqdm
from IPython.core.display import display, HTML
import os
import shutil
import glob 
from pathlib import Path
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer, ColorMode


class CocoCreator:
    # Assumes root_data_dir has both te GT and propagated segmentation labels
    def __init__(self, root_data_dir):
        self.rdd = root_data_dir
        self.segm_dir = os.path.join(root_data_dir, 'seg')
        self.visuals_dir = 'coco_visuals'
        
    def create_coco(self, scene, coco_file_name, pct, use_gt):
        if use_gt:
            self.segm_dir = os.path.join(self.rdd, 'seg_gt')
            self.visuals_dir = 'coco_visuals_gt'
        self.create_metadata()
        self.create_annos(scene, pct)
        self.save_json(coco_file_name)
        self.save_visual_dataset(coco_file_name, scene)
    
    def save_visual_dataset(self, coco_file_name, scene):
        DatasetCatalog.clear()
        MetadataCatalog.clear()

        register_coco_instances('foobar', {}, coco_file_name, os.path.join(self.rdd, 'rgb'))
        MetadataCatalog.get('foobar')
        dataset_dicts = DatasetCatalog.get('foobar')
        
        save_dir = os.path.join(self.rdd, self.visuals_dir)
        print(f'save_dir {save_dir}, coco_file_name {coco_file_name}')
        # if os.path.exists(save_dir):
        #     shutil.rmtree(save_dir)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        for d in dataset_dicts:
            img = cv2.imread(d["file_name"])
            x = d['file_name'].split('/')[-1]
            # print(f"filename {d['file_name'], x}, visual_file {os.path.join(save_dir, x)}")
            visualizer = Visualizer(img[:,:,::-1], metadata=MetadataCatalog.get('foobar'), scale=0.5)
            vis = visualizer.draw_dataset_dict(d)
            img = vis.get_image()
            cv2.imwrite(os.path.join(save_dir, x), img[:,:,::-1])
            # plt.figure(figsize=(4, 4))
            # plt.imshow(img)
            # plt.show()
        
    def save_json(self, coco_file_name):
        coco_output = {
            "info": self.INFO,
            "licenses": self.LICENSES,
            "categories": self.CATEGORIES,
            "images": self.IMAGES,
            "annotations": self.ANNOTATIONS,
        }
        print(f'self.CATS {self.CATEGORIES}')
        print(f"Dumping {len(coco_output['annotations'])} annotations to {coco_file_name}")
        with open(coco_file_name, "w") as output_json:
            json.dump(coco_output, output_json)
        
    def create_metadata(self):
        self.INFO = {}
        self.LICENSES = [{}]
        self.CATEGORIES = []
        self.IMAGES = []
        self.ANNOTATIONS = []
        self.label_to_id = {}
        idc = 1
        for i, label in id_to_label.items():
            instance_label = str(i) + '_' + label
            print(f'instance_label {instance_label}')
            self.CATEGORIES.append({"id": idc, "name": instance_label, "supercategory": "shape"})
            self.label_to_id[i] = idc
            idc += 1
    
    def create_annos(self, scene, pct):
        coco_img_id = -1
        count = 0
        segm_dir = self.segm_dir
        print(f"Scene {scene}, seg dir {segm_dir}")       
        img_dir = os.path.join(self.rdd, 'rgb')
        fs = self.get_segm_files(segm_dir, pct)
        print(f"Creating COCO annotations for {len(fs)} images \n img_dir {img_dir}")
        
        for f in tqdm(fs):
            image_id = int(f.split('.')[0])
            try:
                prop_path = os.path.join(segm_dir, "{:05d}.npy".format(image_id))
                annot = np.load(prop_path).astype(np.uint32)
            except Exception as e:
                print(e)
                continue

            img_filename = "{:05d}.jpg".format(image_id)            
            img = Image.open(os.path.join(img_dir, img_filename))

            # COCO ID and names
            coco_img_id += 1

            image_info = pycococreatortools.create_image_info(
                coco_img_id, os.path.basename(img_filename), img.size
            )

            self.IMAGES.append(image_info)
            for i in np.sort(np.unique(annot.reshape(-1), axis=0)):
                if i in id_to_label:
                    try:
                        category_info = {"id": self.label_to_id[i], "is_crowd": False}
                        # print(f'category_info {category_info}')
                    except Exception as ex:
                        print(f'exception {ex}')
                        continue

                    binary_mask = (annot == i).astype(np.uint32)

                    annotation_info = pycococreatortools.create_annotation_info(
                        count, coco_img_id, category_info, binary_mask, img.size, tolerance=2
                    )
                    if annotation_info is not None:
                        self.ANNOTATIONS.append(annotation_info)
                        count += 1
        
    def get_segm_files(self, segm_dir, pct):
        cs = [os.path.basename(x) for x in glob.glob(os.path.join(segm_dir, '*.npy'))]
        print(f'get_segm_files {segm_dir}, pct {pct}, total_files {len(cs)}')
        cs.sort()
        frq = 1/pct
        fs = []
        for x in range(0, len(cs), int(frq)):
            fs.append(cs[x])
        return fs 

def run_coco(root_data_dir):
    cbase = CocoCreator(root_data_dir)
    cbase.create_coco(
        scene='soumith_apt', 
        coco_file_name=os.path.join(root_data_dir, 'coco.json'),
        pct=1,
        use_gt=False,
    )
    
id_to_label = {
    87: 'plant',
#     97: 'jacket',
    103: 'hamper',
    # 106: 'sidetable',
    140: 'armchair',
}


# build test set - pictures captured from phone 
# run_coco('/home/locobotm/Photos-001-ordered')

# build validation set - pictures from trajectory
# run_coco('/home/locobotm/0512_data/reexplore_final/val_set_sample/')


# build train set 
train_data = '/home/locobotm/0512_data/train_data_1805_3sample/'
for path in Path(train_data).rglob('pred_label*'):
    print(f'running coco for {str(path)}')
    run_coco(str(path))

/tmp/ipykernel_31829/1786034308.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


running coco for /home/locobotm/0512_data/train_data_1805_3sample/s1ppr2/propagated_2/pred_label_p12
instance_label 87_plant
instance_label 103_hamper
instance_label 140_armchair
Scene soumith_apt, seg dir /home/locobotm/0512_data/train_data_1805_3sample/s1ppr2/propagated_2/pred_label_p12/seg
get_segm_files /home/locobotm/0512_data/train_data_1805_3sample/s1ppr2/propagated_2/pred_label_p12/seg, pct 1, total_files 71
Creating COCO annotations for 71 images 
 img_dir /home/locobotm/0512_data/train_data_1805_3sample/s1ppr2/propagated_2/pred_label_p12/rgb


  0%|                                                                                                              | 0/71 [00:00<?, ?it/s]/home/locobotm/miniconda3/envs/droidlet/lib/python3.8/site-packages/pycococreatortools/pycococreatortools.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  contours = np.subtract(contours, 1)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [00:25<00:00,  2.82it/s]


self.CATS [{'id': 1, 'name': '87_plant', 'supercategory': 'shape'}, {'id': 2, 'name': '103_hamper', 'supercategory': 'shape'}, {'id': 3, 'name': '140_armchair', 'supercategory': 'shape'}]
Dumping 54 annotations to /home/locobotm/0512_data/train_data_1805_3sample/s1ppr2/propagated_2/pred_label_p12/coco.json
save_dir /home/locobotm/0512_data/train_data_1805_3sample/s1ppr2/propagated_2/pred_label_p12/coco_visuals, coco_file_name /home/locobotm/0512_data/train_data_1805_3sample/s1ppr2/propagated_2/pred_label_p12/coco.json
running coco for /home/locobotm/0512_data/train_data_1805_3sample/s1ppr2/propagated_2/pred_label_p0
instance_label 87_plant
instance_label 103_hamper
instance_label 140_armchair
Scene soumith_apt, seg dir /home/locobotm/0512_data/train_data_1805_3sample/s1ppr2/propagated_2/pred_label_p0/seg
get_segm_files /home/locobotm/0512_data/train_data_1805_3sample/s1ppr2/propagated_2/pred_label_p0/seg, pct 1, total_files 6
Creating COCO annotations for 6 images 
 img_dir /home/locob

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  4.94it/s]


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:15<00:00,  4.91it/s]


self.CATS [{'id': 1, 'name': '87_plant', 'supercategory': 'shape'}, {'id': 2, 'name': '103_hamper', 'supercategory': 'shape'}, {'id': 3, 'name': '140_armchair', 'supercategory': 'shape'}]
Dumping 41 annotations to /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p12/coco.json
save_dir /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p12/coco_visuals, coco_file_name /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p12/coco.json
running coco for /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p0
instance_label 87_plant
instance_label 103_hamper
instance_label 140_armchair
Scene soumith_apt, seg dir /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p0/seg
get_segm_files /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p0/seg, pct 1, total_files 6
Creating COCO annotations for 6 images 
 img_dir /home/locobotm/0512_dat

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  4.86it/s]


self.CATS [{'id': 1, 'name': '87_plant', 'supercategory': 'shape'}, {'id': 2, 'name': '103_hamper', 'supercategory': 'shape'}, {'id': 3, 'name': '140_armchair', 'supercategory': 'shape'}]
Dumping 8 annotations to /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p0/coco.json
save_dir /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p0/coco_visuals, coco_file_name /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p0/coco.json
running coco for /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p8
instance_label 87_plant
instance_label 103_hamper
instance_label 140_armchair
Scene soumith_apt, seg dir /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p8/seg
get_segm_files /home/locobotm/0512_data/train_data_1805_3sample/r1r2/propagated_2/pred_label_p8/seg, pct 1, total_files 54
Creating COCO annotations for 54 images 
 img_dir /home/locobotm/0512_data/

 69%|█████████████████████████████████████████████████████████████████████▏                               | 37/54 [00:07<00:03,  5.12it/s]

In [11]:
# og_seg_dir = '/home/locobotm/0512_data/reexplore_final/baseline_explore/seg/'

# # create a dict of id to label
# id_to_label = {
#     87: 'plant',
#     97: 'jacket',
#     103: 'hamper',
#     106: 'sidetable',
#     140: 'armchair',
# }

# for x in os.listdir(og_seg_dir):
#     seg = np.load(os.path.join(og_seg_dir, x)).astype(np.uint32)
#     seg[seg == True] = int(x.split('.')[0])
#     print(x, np.unique(seg))
#     np.save(os.path.join(og_seg_dir, x), seg)

00140.npy [  0 140]
00106.npy [  0 106]
00087.npy [ 0 87]
00097.npy [ 0 97]
00103.npy [  0 103]
